# strides: Why are some things fast, and other things slow?

For small data sets (< 1MB), modern computers are so amazingly fast that efficiency is hardly worth thinking about.
But if you are using NumPy, it is not uncommon to have bigger data sets that actually take some time to process, even on speedy machines.
For example, in the previous section, we allocated data arrays of 800MB and it took a couple of seconds to process it.
With that kind of data, speed starts to matter.

Understanding the kind of operations that are expensive (take a long time) and which ones are cheap can be surprisingly hard when it comes to NumPy.
A big part of data processing speed is memory management.
Copying big arrays takes time, so the less of that we do, the faster our code runs.
The rules of when NumPy copies data are not trivial and it is worth your while to take a closer look at them.
This involves developing an understanding of how NumPy's `ndarray` datastructure works behind the scenes.

## An example: matrix transpose

Transposing a matrix means that all rows become columns and all columns become rows. All off-diagonal values change places. Obviously, all this moving data around should take come time, right? Let's see how long NumPy's transpose function takes, by transposing a huge (10 000 ✕ 20 000) matrix. This time, let's exclude the time it takes to allocate this beast of a matrix and focus solely on the [np.transpose](https://docs.scipy.org/doc/numpy/reference/generated/numpy.transpose.html) function:

In [ ]:
import numpy as np
np.random.seed(0)
a = np.random.rand(10_000, 20_000)
print('Matrix `a` takes up %d MB' % (a.nbytes / 10**6))

In [ ]:
%%timeit -r 10 -n 1000
b = a.transpose()

<img src="images/magic.jpg" style="width: 200px; float: right; margin: 10px; margin-top: -10px"/>

On my laptop, it takes mere nanoseconds to transpose 1600 MB of data.
NumPy somehow avoided copying the data.
The transpose operation is computationally so cheap that is also available through the `.T` property (it is conventially not done to expose methods that are potentially computationally expensive as properties).

What black magic is going on here?
To answer that, we must dive a little into the innards of NumPy `ndarray` datastructure.

## The `ndarray` exposed

The first thing you need to know about `ndarray` is that the memory backing up the `ndarray` is always a flat 1D array.
That is how the operating system exposes memory. (This is a lie by the way, the actual memory is a fragmented mess, but the operating system shelters us from this cold harsh truth.)
For example, a 2D matrix is stored with all the rows concatenated as a single long vector.

<img src="images/memory_layout.png" width="700">

NumPy is faking the second dimension behind the scenes!
When we request the element at say, `[2, 3]`, NumPy converts this to the correct index in the long 1D array `[11]`.
Converting `[2, 3]` into `[11]` is called "raveling", and the reverse "unraveling".

The implications of this are many, so take let's take some time to understand it properly by writing our own `ravel()` function that converts a row and column index to the appropriate index in the 1D array.
Use the image above as a guide. We'll discuss the answer below, but do youself a favor and don't peek before you've written the function correctly yourself.

In [ ]:
def ravel(row_index, col_index, n_rows, n_cols):
    """Ravel a row/col matrix index to a flat vector index.
    
    Parameters
    ----------
    row_index : int
        The row of the requested element in the matrix.
    col_index : int
        The column of the requested element in the matrix.
    n_rows : int
        The total number of rows in the matrix.
    n_cols : int
        The total number of columns in the matrix.
        
    Returns
    -------
    index : int
        The corresponding index in a flat 1D array.
    """
    return  # Do the computation here

# If you wrote the function correctly, these tests should all pass
assert ravel(2, 3, n_rows=4, n_cols=4) == 11
assert ravel(2, 3, n_rows=4, n_cols=8) == 19
assert ravel(0, 0, n_rows=1, n_cols=1) == 0
assert ravel(3, 3, n_rows=4, n_cols=4) == 15
assert ravel(3_465, 18_923, n_rows=10_000, n_cols=20_000) == 69_318_923

I hope that, after a bit of trial and error, you managed to get this right.
To get to the next row, we have to skip over `n_cols` indices.
To get to the next column, we can just add `1`.
What would it take to generalize this code to work with an arbitrary number of dimensions?

NumPy solves this through the concept of "strides":

In [ ]:
a = np.zeros((4, 8))  # Create a 4 x 8 matrix
a.strides

The result above tells us that to get to the next row in this matrix, we have to skip ahead 64 bytes.
64? Yes!
We have created a matrix consisting of double-precision floating point numbers.
Each one of those bad boys takes up 8 bytes, so all the indices are multiplied by 8 to get to the proper byte in the memory array.
To move to the next column in the matrix, we skip ahead 8 bytes.

What about a 5D `ndarray`?

In [ ]:
a = np.zeros((4, 5, 6, 7, 8))
a.strides

The `.strides` attribute contains for each dimension, the number of bytes we have to skip over to get to the next element along that dimension.

## The mystery of `transpose()` revealed

So how does NumPy manage to implement `transpose()` so efficiently?
Look what happens to the `.strides`:

In [ ]:
a = np.random.rand(10_000, 20_000)
print(f'Normal: {a.shape=} {a.strides=}')
a = a.T
print(f'Transposed: {a.shape=} {a.strides=}')

It just flipped the `.strides`!
The huge 1D array containing the data is untouched.
Normally, a (20 000 ✕ 10 000) matrix would have strides `(80_000, 8)`:

In [ ]:
b = np.random.rand(20_000, 10_000)
b.strides

Yet, our transposed `a.T` matrix has strides `(8, 160_000)`, so it did't have to move any data around.

A little known feature of NumPy is the `stride_tricks` module that allows you to modify the `.strides` attribute directly.
Playing around with this is very educational:

In [ ]:
from numpy.lib.stride_tricks import as_strided

def transpose(a):
    """My own transpose function"""
    # We just flip the .strides!
    return as_strided(a, shape=a.shape[::-1], strides=a.strides[::-1])

# Test the function on a small matrix
a = np.array([[1, 2, 3],
              [4, 5, 6]])
print('Before transpose:')
print(a)
print('After transpose:')
print(transpose(a))

## Surely, `reshape` is fast

Knowing about `.strides`, it should not be a surprise that changing the shape of an array through `reshape()` can be accomplished without any copying of data:

In [ ]:
a = np.random.rand(20_000, 10_000)
print(f'{a.strides=}')
b = a.reshape(40_000, 5_000)
print(f'{b.strides=}')
c = a.reshape(20_000, 5_000, 2)
print(f'{c.strides=}')

For fun and giggles, I challenge you to use `as_strided` to create a (5 ✕ 100 000 000 000) array containing on the first row all `1`'s, the second row all `2`'s, etc:

```
array([[1., 1., 1., ..., 1., 1., 1.],
       [2., 2., 2., ..., 2., 2., 2.],
       [3., 3., 3., ..., 3., 3., 3.],
       [4., 4., 4., ..., 4., 4., 4.],
       [5., 5., 5., ..., 5., 5., 5.]])
```

In [ ]:
a = np.array([1., 2., 3., 4., 5.])
a = as_strided(a, shape=..., strides=...)  # Fill in the correct shape and strides

# If you got it right, these tests should pass
assert a.shape == (5, 100_000_000_000)
assert a[0, 23890493] == 1.
assert a[3, 19302839] == 4.

## A fast thing + a fast thing = a fast thing?

If `transpose()` is fast, and `reshape()` is fast, then doing them both must be fast too, right?

In [ ]:
a = np.random.rand(10_000, 20_000)

In [ ]:
%%timeit -n 1 -r 1
a.T.reshape(40_000, 5_000)

Wrong! In this case, the data actually had to be copied and it's super slow (it takes *seconds* instead of nanoseconds).
Why?
When an `ndarray` is first created, it is laid out in memory row-by-row (see image above).
This is also how the programming language C does it, so this is called `C_CONTIGUOUS`.
The transpose left the data laid out in memory column-by-column.
This is also how the programming language Fortran does it, so this is called `F_CONTIGUOUS`.

In [ ]:
print(a.flags['C_CONTIGUOUS'], a.flags['F_CONTIGUOUS'])
print(a.T.flags['C_CONTIGUOUS'], a.T.flags['F_CONTIGUOUS'])

To see why the copying of data was inevitable, look at what happens to this smaller (2 ✕ 3) matrix after transposition and reshaping. You van verify for yourself there is no way to get the final array based on the first array and some clever setting of the `.strides`.

In [ ]:
a = np.array([[1, 2, 3], [4, 5, 6]])

print('Original array:')
print(a)

print('\nTransposed:')
print(a.T)

print('\nTransposed and then reshaped:')
print(a.T.reshape(2, 3))

<div style="border: 3px solid #aaccff; margin: 10px 100px; padding: 10px">
    <b>Main takeaway of this chapter</b>

NumPy will avoid copying memory whenever it can. Whether it can depends on what kind of layout the data is currently in.
</div>

In the following chapter, we will look at another way NumPy avoids copying data, namely through "views".

<a href="views.ipynb" style="font-size: 20px;">Continue to next chapter</a>